In [68]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
#from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
#from sklearn.model_selection import train_test_split
from skimage import io
import os
import scipy.misc
from scipy.misc import imread, imresize
from keras import regularizers
import csv

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("../input/fruits-360_dataset"))
print(os.listdir("../input/fruits-360_dataset/fruits-360"))
# Any results you write to the current directory are saved as output.

['fruits-360_dataset']
['fruits-360']
['test-multiple_fruits', 'readme.md', 'Test', 'LICENSE', 'papers', 'Training']


Function to get training and validation images.

In [70]:
import cv2

def load_images(path):
    img_data = []
    labels = []
    idx_to_label = []
    i = -1
    for fruit in os.listdir(path):
        fruit_path = os.path.join(path,fruit)
        labels.append(fruit)
        i = i+1
        for img in os.listdir(fruit_path):
            img_path = os.path.join(fruit_path,img)
            image = cv2.imread(img_path)
            image = cv2.resize(image, (64, 64))
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            img_data.append(image)
            idx_to_label.append(i)
    return np.array(img_data),np.array(idx_to_label),labels

#### Path Directories
The following code gives us a list of all directories contained in our path.

- 'Training' : Contains all our training Images.
- 'Test' Contains all our Test or validation images.
Please note that inside Training/Validation folders all classes of images are separated into different folders.

Calling the function.

In [71]:
trn_data_path = '../input/fruits-360_dataset/fruits-360/Training'
val_data_path = '../input/fruits-360_dataset/fruits-360/Test'
X_train,y_train,label_data = load_images(trn_data_path)
print(os.listdir("../input/fruits-360_dataset/fruits-360"))
print(X_train.shape)
print(y_train.shape)
print(label_data)
X_test,y_test,label_data_garbage = load_images(val_data_path)
print(y_test.shape)

['test-multiple_fruits', 'readme.md', 'Test', 'LICENSE', 'papers', 'Training']
(41322, 64, 64, 3)
(41322,)
['Nectarine', 'Kaki', 'Avocado', 'Apple Golden 3', 'Lemon', 'Pear Monster', 'Apple Red Yellow', 'Tomato 1', 'Cherry 2', 'Apple Red 1', 'Pear Abate', 'Mandarine', 'Strawberry', 'Cherry Rainier', 'Cocos', 'Cantaloupe 2', 'Granadilla', 'Pomegranate', 'Lychee', 'Quince', 'Raspberry', 'Tangelo', 'Mango', 'Plum', 'Tamarillo', 'Apple Granny Smith', 'Tomato Maroon', 'Passion Fruit', 'Lemon Meyer', 'Grapefruit White', 'Banana Red', 'Tomato 4', 'Huckleberry', 'Cherry Wax Yellow', 'Physalis with Husk', 'Dates', 'Clementine', 'Strawberry Wedge', 'Pitahaya Red', 'Cherry Wax Red', 'Limes', 'Carambula', 'Tomato 2', 'Apple Red 2', 'Peach', 'Apple Golden 2', 'Cherry 1', 'Tomato 3', 'Physalis', 'Tomato Cherry Red', 'Pepino', 'Maracuja', 'Pear Williams', 'Walnut', 'Pineapple', 'Avocado ripe', 'Papaya', 'Pear', 'Cherry Wax Black', 'Cactus fruit', 'Rambutan', 'Apple Red Delicious', 'Pineapple Mini', '

One hot encoding.

In [72]:
num_of_classes = 81
Y_train = np_utils.to_categorical(y_train, num_of_classes)
Y_test = np_utils.to_categorical(y_test, num_of_classes)

Model. I played around with the dropout values and achieved better score at 0.7 activations can be Relu, elu, LeakyRelu.

In [73]:
model = Sequential()

model.add(Conv2D(16, (3, 3), input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3 )))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())

# Fully connected layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.7))
model.add(Dense(81))

model.add(Activation('softmax'))

In [74]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 62, 62, 16)        448       
_________________________________________________________________
batch_normalization_25 (Batc (None, 62, 62, 16)        64        
_________________________________________________________________
activation_29 (Activation)   (None, 62, 62, 16)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 60, 60, 16)        2320      
_________________________________________________________________
batch_normalization_26 (Batc (None, 60, 60, 16)        64        
_________________________________________________________________
activation_30 (Activation)   (None, 60, 60, 16)        0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 30, 30, 16)        0         
__________

In [75]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(),metrics=['accuracy'])

No augmentation.

In [76]:
model.fit(X_train,Y_train, epochs=1,batch_size = 32,
                    validation_data=(X_test,Y_test))

Train on 41322 samples, validate on 13877 samples
Epoch 1/1
41322/41322 [==============================] - 36s 861us/step - loss: 0.4600 - acc: 0.8792 - val_loss: 0.1960 - val_acc: 0.9455


In [77]:
model.fit(X_train,Y_train, epochs=1,batch_size = 32,
                    validation_data=(X_test,Y_test))

Train on 41322 samples, validate on 13877 samples
Epoch 1/1
41322/41322 [==============================] - 34s 811us/step - loss: 0.0943 - acc: 0.9718 - val_loss: 0.2476 - val_acc: 0.9191


In [78]:
model.fit(X_train,Y_train, epochs=3,batch_size = 32,
                    validation_data=(X_test,Y_test))

Train on 41322 samples, validate on 13877 samples
Epoch 1/3
41322/41322 [==============================] - 33s 811us/step - loss: 0.0688 - acc: 0.9790 - val_loss: 0.1010 - val_acc: 0.9653
Epoch 2/3
41322/41322 [==============================] - 33s 805us/step - loss: 0.0554 - acc: 0.9823 - val_loss: 0.0707 - val_acc: 0.9803
Epoch 3/3
41322/41322 [==============================] - 33s 807us/step - loss: 0.0535 - acc: 0.9819 - val_loss: 0.1057 - val_acc: 0.9677


In [79]:
predictions = model.predict_classes(X_test)

predictions = list(predictions)
actuals = list(y_test)

sub = pd.DataFrame({'Actual': actuals, 'Predictions': predictions})
sub.to_csv('./output_cnn.csv', index=False)

Augmentation.

In [80]:
gen = ImageDataGenerator(width_shift_range=.2, 
                             height_shift_range=.2,
                         zoom_range = 0.1,
                        horizontal_flip = 'True')

test_gen = ImageDataGenerator()
train_generator = gen.flow(X_train, Y_train, batch_size=32)
test_generator = test_gen.flow(X_test, Y_test, batch_size=32)

In [81]:
score = model.evaluate(X_test, Y_test)
print()
print('Test loss: ', score[0])
print('Test Accuracy', score[1])

13877/13877 [==============================] - 3s 214us/step

Test loss:  0.10573129625911125
Test Accuracy 0.9677163652086186
